## Preparing Embeddings for Narrative Maps

In [1]:
import pandas as pd
from ftfy import fix_encoding
from datetime import date, datetime, timezone
from sentence_transformers import SentenceTransformer
import numpy as np
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [2]:
dataset = "cv"
query = pd.read_csv(dataset + ".csv")
# Cleaning titles, might need to add more pre-processing for dirtier data sets.
query['title'] = query['title'].map(lambda a: fix_encoding(a))
query['title'] = query['title'].map(lambda a: a.replace("’", "'").replace("‘", "'"))
for index, article in query.iterrows():
    # Fixing encoding issues.
    title = fix_encoding(query.loc[index,'title'])
    full_text = fix_encoding(query.loc[index,'full_text'])
    # Fixing some additional characters that cause issues in display.
    title = title.replace("’", "'").replace("‘", "'").replace("–","-").replace(" "," ").replace("—","-").replace("&amp","and") 
    full_text = full_text.replace("’", "'").replace("‘", "'").replace("–","-").replace(" "," ").replace("—","-").replace("&amp","and") 
    # News specific stufff
    query.loc[index,'title'] = title.replace("- " + query.loc[index,'publication'],"").replace("- The New York Times","").replace("| TheHill","")

print("Embedding...")
corpus_embeddings = bert_model.encode(query['title'] + "\n" + query['full_text'], convert_to_tensor=True).cpu()
print("Converting to np...")
main_array = corpus_embeddings.numpy()
print("Converting to list...")
query['embed'] = list(main_array)
query.to_csv(dataset + "_final.csv", index=False)    

Embedding...


KeyboardInterrupt: 